# Vehicles Detection Using YoLo V8

Importing usefull libraries

In [2]:
import ultralytics
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from ultralytics import YOLO

from tracker import *

In [3]:
ultralytics.__version__

'8.2.25'

Initializing the model

In [4]:
Model=YOLO("yolov8s.pt")

COCO dataset class list

In [5]:
class_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


settin up tracker and count 

In [6]:
tracker = Tracker()
count=0

In [7]:
vid=cv2.VideoCapture("Y2meta.app-Test Video for Vehicle Counting Model - Indian Road-(1080p).mp4")

Playing Video

In [8]:

def play_video(vid):    
    if (vid.isOpened()== False): 
        print("Error opening video file") 
    
    while(vid.isOpened()): 
        
        ret, frame = vid.read() 
        if ret == True: 
            cv2.imshow('Frame', frame) 
            
            if cv2.waitKey(25) & 0xFF == ord('k'): 
                break
    
        else: 
            break
    
    # vid.release() 
    

In [9]:
play_video(vid)

In [10]:
vid=cv2.VideoCapture("Y2meta.app-Test Video for Vehicle Counting Model - Indian Road-(1080p).mp4")


Model

In [11]:
up_cars={}
down_cars={}
counter_up_cars=[]
counter_down_cars=[]

up_truck={}
down_truck={}
counter_up_truck=[]
counter_down_truck=[]

up_bus={}
down_bus={}
counter_up_bus=[]
counter_down_bus=[]

up_motorcycle={}
down_motorcycle={}
counter_up_motorcycle=[]
counter_down_motorcycle=[]

while True:
    ret,frame=vid.read()
    if not ret:
        print("video not available")
        break
    count+=1
    frame=cv2.resize(frame,(1020,500))
    
    results=Model.predict(frame)
    # print(results)
    a=results[0].boxes.data
    a=a.detach().cpu().numpy()
    px=pd.DataFrame(a).astype("float")
    print(px)
    list_car=[]
    list_truck=[]
    list_bus=[]
    list_motorcycle=[]
    
    
    for index,row in px.iterrows():
        x1=int(row[0])
        y1=int(row[1])
        x2=int(row[2])
        y2=int(row[3])
        d=int(row[5])
        c=class_list[d]
        if 'car' in c:
            list_car.append([x1,y1,x2,y2])
        elif 'bus' in c:
            list_bus.append([x1,y1,x2,y2])

        elif 'truck' in c:
            list_truck.append([x1,y1,x2,y2])

        elif 'motorcycle' in c:
            list_motorcycle.append([x1,y1,x2,y2])
    
    bbox_id=tracker.update(list_car)
    for bbox in bbox_id:
        x3,y3,x4,y4,id=bbox
        cx=(x3+x4)//2
        cy=(y3+y4)//2
        # ! 2 different lines to detect movement 
        line_y_1=270 
        line_y_2=200
        offset = 5
        
        # cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
        # cv2.putText(frame,str(id),(cx,cy),cv2.FONT_HERSHEY_COMPLEX,0.8,(255,255,255),2)
        
        if line_y_2 < (cy + offset) and line_y_2 > (cy - offset):
            down_cars[id]=cy   
        if id in down_cars:
            if line_y_1 < (cy + offset) and line_y_1 > (cy - offset):         
                cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
                cv2.putText(frame,str(id),(cx,cy),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,255,255),2)
                #counter++
                counter_down_cars.append(id)  


        if line_y_1 < (cy + offset) and line_y_1 > (cy - offset):
            up_cars[id]=cy   
        if id in up_cars:
            if line_y_2 < (cy + offset) and line_y_2> (cy - offset):         
                cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
                cv2.putText(frame,str(id),(cx,cy),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,255,255),2)
                #counter++
                counter_up_cars.append(id)
                
    # ! bus
    bbox_id_bus=tracker.update(list_bus)  
    for bbox in bbox_id_bus:
        x3,y3,x4,y4,id=bbox
        cx=(x3+x4)//2
        cy=(y3+y4)//2
        # ! 2 different lines to detect movement 
        line_y_1=270 
        line_y_2=200
        offset = 5
        
        cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
        cv2.putText(frame,str(id),(cx,cy),cv2.FONT_HERSHEY_COMPLEX,0.8,(255,255,255),2)
        
        if line_y_2 < (cy + offset) and line_y_2 > (cy - offset):
            down_bus[id]=cy   
        if id in down_bus:
            if line_y_1 < (cy + offset) and line_y_1 > (cy - offset):         
                cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
                cv2.putText(frame,str(id),(cx,cy),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,255,255),2)
                #counter++
                counter_down_bus.append(id)  


        if line_y_1 < (cy + offset) and line_y_1 > (cy - offset):
            up_bus[id]=cy   
        if id in up_bus:
            if line_y_2 < (cy + offset) and line_y_2> (cy - offset):         
                cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
                cv2.putText(frame,str(id),(cx,cy),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,255,255),2)
                #counter++
                counter_up_bus.append(id)
        
    
    bbox_id_bus=tracker.update(list_bus)  
    for bbox in bbox_id_bus:
        x3,y3,x4,y4,id=bbox
        cx=(x3+x4)//2
        cy=(y3+y4)//2
        # ! 2 different lines to detect movement 
        line_y_1=270 
        line_y_2=200
        offset = 5
        
        cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
        cv2.putText(frame,str(id),(cx,cy),cv2.FONT_HERSHEY_COMPLEX,0.8,(255,255,255),2)
        
        if line_y_2 < (cy + offset) and line_y_2 > (cy - offset):
            down_bus[id]=cy   
        if id in down_bus:
            if line_y_1 < (cy + offset) and line_y_1 > (cy - offset):         
                cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
                cv2.putText(frame,str(id),(cx,cy),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,255,255),2)
                #counter++
                counter_down_bus.append(id)  


        if line_y_1 < (cy + offset) and line_y_1 > (cy - offset):
            up_bus[id]=cy   
        if id in up_bus:
            if line_y_2 < (cy + offset) and line_y_2> (cy - offset):         
                cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
                cv2.putText(frame,str(id),(cx,cy),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,255,255),2)
                #counter++
                counter_up_bus.append(id)
                
    # ! Trucks
    bbox_id_truck=tracker.update(list_truck)  
    for bbox in bbox_id_truck:
        x3,y3,x4,y4,id=bbox
        cx=(x3+x4)//2
        cy=(y3+y4)//2
        # ! 2 different lines to detect movement 
        line_y_1=270 
        line_y_2=200
        offset = 5
        
        cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
        cv2.putText(frame,str(id),(cx,cy),cv2.FONT_HERSHEY_COMPLEX,0.8,(255,255,255),2)
        
        if line_y_2 < (cy + offset) and line_y_2 > (cy - offset):
            down_truck[id]=cy   
        if id in down_truck:
            if line_y_1 < (cy + offset) and line_y_1 > (cy - offset):         
                cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
                cv2.putText(frame,str(id),(cx,cy),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,255,255),2)
                #counter++
                counter_down_truck.append(id)  


        if line_y_1 < (cy + offset) and line_y_1 > (cy - offset):
            up_truck[id]=cy   
        if id in up_truck:
            if line_y_2 < (cy + offset) and line_y_2> (cy - offset):         
                cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
                cv2.putText(frame,str(id),(cx,cy),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,255,255),2)
                #counter++
                counter_up_truck.append(id)
    # ! motorcycle
    bbox_id_motorcycle=tracker.update(list_motorcycle)  
    for bbox in bbox_id_motorcycle:
        x3,y3,x4,y4,id=bbox
        cx=(x3+x4)//2
        cy=(y3+y4)//2
        # ! 2 different lines to detect movement 
        line_y_1=270 
        line_y_2=200
        offset = 5
        
        cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
        cv2.putText(frame,str(id),(cx,cy),cv2.FONT_HERSHEY_COMPLEX,0.8,(255,255,255),2)
        
        if line_y_2 < (cy + offset) and line_y_2 > (cy - offset):
            down_motorcycle[id]=cy   
        if id in down_motorcycle:
            if line_y_1 < (cy + offset) and line_y_1 > (cy - offset):         
                cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
                cv2.putText(frame,str(id),(cx,cy),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,255,255),2)
                #counter++
                counter_down_motorcycle.append(id)  


        if line_y_1 < (cy + offset) and line_y_1 > (cy - offset):
            up_motorcycle[id]=cy   
        if id in up_motorcycle:
            if line_y_2 < (cy + offset) and line_y_2> (cy - offset):         
                cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
                cv2.putText(frame,str(id),(cx,cy),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,255,255),2)
                #counter++
                counter_up_motorcycle.append(id)
    
    
    # ! line 
    text_color = (255,255,255) 
    red_color = (0, 0, 255)   
    blue_color = (255, 0, 0)  
    green_color = (0, 255, 0)
    
    cv2.line(frame,(8,270),(930,270),blue_color,3)  
    cv2.putText(frame,('line'),(8,270),cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA) 
    
    downwards = (len(counter_down_cars))
    
    cv2.putText(frame,('going down - cars :')+ str(downwards),(60,40),cv2.FONT_HERSHEY_COMPLEX, 0.5, green_color, 1, cv2.LINE_AA)    

    upwards = (len(counter_up_cars))
    cv2.putText(frame,('going up - cars :')+ str(upwards),(60,60),cv2.FONT_HERSHEY_COMPLEX,0.5, text_color, 1, cv2.LINE_AA) 
    
    downwards_bus = (len(counter_down_bus))
    
    cv2.putText(frame,('going down - buses :')+ str(downwards_bus),(60,80),cv2.FONT_HERSHEY_COMPLEX, 0.5, green_color, 1, cv2.LINE_AA)    

    upwards_bus = (len(counter_up_bus))
    cv2.putText(frame,('going up - buses :')+ str(upwards_bus),(60,100),cv2.FONT_HERSHEY_COMPLEX,0.5, text_color, 1, cv2.LINE_AA) 
    
    downwards_truck = (len(counter_down_truck))
    
    cv2.putText(frame,('going down - truck :')+ str(downwards_truck),(60,120),cv2.FONT_HERSHEY_COMPLEX, 0.5, green_color, 1, cv2.LINE_AA)    

    upwards_truck = (len(counter_up_truck))
    cv2.putText(frame,('going up - truck :')+ str(upwards_truck),(60,140),cv2.FONT_HERSHEY_COMPLEX,0.5, text_color, 1, cv2.LINE_AA)
    
    downwards_motorcycle = (len(counter_down_motorcycle))
    
    cv2.putText(frame,('going down - motorcycle :')+ str(downwards_motorcycle),(60,160),cv2.FONT_HERSHEY_COMPLEX, 0.5, green_color, 1, cv2.LINE_AA)    

    upwards_motorcycle = (len(counter_up_motorcycle))
    cv2.putText(frame,('going up - motorcycle :')+ str(upwards_motorcycle),(60,180),cv2.FONT_HERSHEY_COMPLEX,0.5, text_color, 1, cv2.LINE_AA)
    
    
    cv2.imshow("frames",frame)
    
    if cv2.waitKey(1)&0xFF==27:
        break
vid.release()
cv2.destroyAllWindows()


0: 320x640 6 persons, 7 cars, 4 motorcycles, 597.7ms
Speed: 7.6ms preprocess, 597.7ms inference, 2575.0ms postprocess per image at shape (1, 3, 320, 640)
             0           1            2           3         4    5
0   518.563293  230.869263   595.473633  304.131073  0.861554  2.0
1   332.212952  136.364868   369.927368  167.782181  0.763232  2.0
2   756.626648  330.965668   801.144775  393.920837  0.597404  3.0
3   207.739182  180.485367   224.390396  214.690582  0.593021  0.0
4   480.108215   92.001289   501.707306  108.674942  0.586352  2.0
5   758.295959  313.429993   798.640625  367.044373  0.577651  0.0
6   514.133972   92.306732   535.973145  109.213387  0.460799  2.0
7   189.822433  186.109436   200.931732  215.501694  0.435454  0.0
8   855.774292  251.284805   968.025452  315.247040  0.405265  3.0
9   576.742554  131.176910   591.774048  153.239822  0.386580  0.0
10  352.188171   79.391129   378.945251  103.809479  0.383197  2.0
11  992.424927  269.169037  1019.521545  

KeyboardInterrupt: 